In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer

import pandas as pd
import numpy as np
import scipy as sp



In [43]:
df = pd.read_csv('salary-train.csv')
test = pd.read_csv('salary-test-mini.csv')
#преобработка
df['FullDescription'] = df['FullDescription'].str.lower()
df['LocationNormalized'] = df['LocationNormalized'].str.lower()

df['FullDescription'] = df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)


                                     FullDescription LocationNormalized  \
0  international sales manager london     k      ...             london   
1  an ideal opportunity for an individual that ha...             london   
2  online content and brand manager   luxury reta...  south east london   
3  a great local marketleader is seeking a perman...            dereham   
4  registered nurse   rgn  nursing home for young...   sutton coldfield   

  ContractTime  SalaryNormalized  
0    permanent             33000  
1    permanent             50000  
2    permanent             40000  
3    permanent             22500  
4          NaN             20355  


In [45]:
#transform to vectors
vectoriser = TfidfVectorizer(min_df=5)
x1_train = vectoriser.fit_transform(df['FullDescription'])
x1_test = vectoriser.transform(test['FullDescription'])


In [53]:
df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)
test['LocationNormalized'].fillna('nan', inplace=True)
test['ContractTime'].fillna('nan', inplace=True)




In [54]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [55]:
new_df = sp.sparse.hstack([x1_train, X_train_categ])
test_df = sp.sparse.hstack([x1_test, X_test_categ])

In [58]:
ridge = Ridge(alpha=1, random_state=241)
ridge.fit(new_df, df['SalaryNormalized'])
predictions = ridge.predict(test_df)
print(predictions)

[55850.5934512  37182.51069003]
